# This is the same as *Policy Evaluation Solution.ipynb*, just that contains detailed explanation of each line or couple of lines and a couple of extra prints to visualize better the process

In these DP problems, we'll assume complete knowledge of the environment dynamics. I.e, we can predict what will happen if we do action 'a' on every state 's' from the beginning. 

In [58]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv
# Importing gridworld environment. It's a modification of Open AI Gym's Toy_Text "Discrete", following
# Suttons example from chapter 4 (pdf pg. 100) of his Reinforcement Learning book.
# Original: https://github.com/openai/gym/blob/master/gym/envs/toy_text/discrete.py

Sutton's *Reinforcement Learning: An Introduction*: https://webdocs.cs.ualberta.ca/~sutton/book/bookdraft2016sep.pdf

Original env: https://github.com/openai/gym/blob/master/gym/envs/toy_text/discrete.py

Gridworld env: https://github.com/dennybritz/reinforcement-learning/blob/master/lib/envs/gridworld.py

In [59]:
#pp = pprint.PrettyPrinter(indent=2)
# I think it's not used here
env = GridworldEnv()
# now env is our gridworld environment

![Gridworld-from-Book](../images/DP-PE-Gridworld.png)

## Iterative Policy Evaluation
### Pseudocode (copy-paste from the book)

Input $\pi$, the policy to be evaluated

Initialize an array $V(s)=0$, for all $s\in S^+$

Repeat

. . . . $\Delta \leftarrow 0$
    
. . . . For each $s\in S$:
        
. . . . . . . . $v \leftarrow V(s)$
        
. . . . . . . . $V(s) \leftarrow \sum_{a} \pi(a|s) \sum_{s',r} p(s',r|s,a)[r + \gamma V(s')]$
        
. . . . . . . . $\Delta \leftarrow max( \Delta, \left|v - V (s)\right| )$
        
until $\Delta < \theta$ (a small positive number)
    
Output $V \approx v_\pi$

![Gridworld-from-Book](../images/DP-Policy-Evaluation.png)

### Origin of V(s) function:

$V_{ \pi } (s) = \sum_{a} \pi (a|s) * q_\pi (s,a)$

$q_{\pi } (s,a) = R_a^s + \gamma \sum_{s'\in S} P_{ss'}^a v_\pi (s)$

$V(s) = \sum_{a} \pi (a|s) \sum_{s',r} p(s',r|s,a)[r + \gamma V(s')]$

For a more detailed and clear explanation, watch *RL Course by David Silver - Lecture 2: Markov Decision Process* @ 53m36s
https://youtu.be/lfHX2hHRMVQ?t=53m36s

In [60]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    # Initialize an array V(s)=0, for all s∈S+
    # It's going to save a value for every state as a vector here
    V = np.zeros(env.nS)
    i = 0
    # Setting decimal points to better visualizing
    np.set_printoptions(precision=2)
    print("Grid Value Function at iteration: ", i)
    print(np.array(V).reshape(env.shape))
    print("")
    while True:
        # ∆ ← 0
        delta = 0
        i += 1
        # For each state, perform a "full backup"
        # For each s∈S:
        for s in range(env.nS):
            # from position 0 to 15 of the grid
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # 'a' can be 0, 1, 2 or 3
                # with random_policy, action_prob is always 0.25 for each 'a'
                # For each action, look at the possible next states...
                # [s] will be position in grid
                # [a] will be action taked
                for  prob, next_state, reward, done in env.P[s][a]:
                    # for example, if in position 0 of grid, every action will lead to terminal state
                    # eg. if in position 1 of grid:
                    #     a=0(up) will end up on same position with 100% chance
                    #     a=1(right) will end up on grid position 2 with 100% chance
                    #     a=2(down) will end up on grid position 5 (below pos.1) with 100% chance
                    #     a=3(left) will end up on grid position 0 with 100% chance, which is terminal
                    # each transition but terminal state has a reward of -1
                    #print("s:",s, "a:",a, "prob:", prob, "next:", next_state, "reward:", reward, "done:",done)
                    # Calculate the expected value
                    # V(s) ← Sum_{a} π(a|s) Sum_{s',r} p(s',r|s,a)[r + γV(s')]
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
                    # it saves the expected value in v instead of V(s), so it can sum over actions and
                    # next_states and rewards, but later copy v into V[s] and repeat for every state/grid position
            # How much our value function changed (across any states)
            # ∆ ← max( ∆, |v − V (s)| )
            delta = max(delta, np.abs(v - V[s]))
            # Now copies v into V(s)
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        # until ∆ < θ (a small positive number)
        if delta < theta:
            break
        if  i==1 or i%20 == 0:
            print("Grid Value Function at iteration: ", i)
            print(np.array(V).reshape(env.shape))
            print("delta: ", delta)
            print("")
    #Output V ≈ v_π
    return np.array(V)

In [61]:
#Input π, the policy to be evaluated
random_policy = np.ones([env.nS, env.nA]) / env.nA
# value of policy
v = policy_eval(random_policy, env)

Grid Value Function at iteration:  0
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

Grid Value Function at iteration:  1
[[ 0.   -1.   -1.25 -1.31]
 [-1.   -1.5  -1.69 -1.75]
 [-1.25 -1.69 -1.84 -1.9 ]
 [-1.31 -1.75 -1.9   0.  ]]
delta:  1.8984375

Grid Value Function at iteration:  20
[[  0.   -11.43 -16.3  -17.93]
 [-11.43 -14.84 -16.57 -16.61]
 [-16.3  -16.57 -15.11 -11.84]
 [-17.93 -16.61 -11.84   0.  ]]
delta:  0.372594031843

Grid Value Function at iteration:  40
[[  0.   -13.55 -19.36 -21.29]
 [-13.55 -17.45 -19.4  -19.41]
 [-19.36 -19.4  -17.5  -13.62]
 [-21.29 -19.41 -13.62   0.  ]]
delta:  0.0647083950673

Grid Value Function at iteration:  60
[[  0.   -13.92 -19.89 -21.88]
 [-13.92 -17.9  -19.9  -19.9 ]
 [-19.89 -19.9  -17.91 -13.93]
 [-21.88 -19.9  -13.93   0.  ]]
delta:  0.0112378558487

Grid Value Function at iteration:  80
[[  0.   -13.99 -19.98 -21.98]
 [-13.99 -17.98 -19.98 -19.98]
 [-19.98 -19.98 -17.98 -13.99]
 [-21.98 -19.98 -13.99   0

In [62]:
print("Value Function:")
print(v)
print("")

#Grid values. Compare to image.
print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value Function:
[  0. -14. -20. -22. -14. -18. -20. -20. -20. -20. -18. -14. -22. -20. -14.
   0.]

Reshaped Grid Value Function:
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]



![Gridworld-OptimalPolicy-from-Book](../images/DP-PE-OptimalPolicy.png)

In [63]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)